In [ ]:
from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter
from lifelines.statistics import logrank_test
import pandas as pd
import matplotlib.pyplot as plt
import warnings; warnings.filterwarnings(action='ignore')

In [ ]:
df = pd.read_csv("/path of predicted results from machine learning model.csv/",header=0,index_col=0)

kmf = KaplanMeierFitter(alpha = 0.95)   #alpha设置置信区间

T = df['Duration'] / 365                       #生存时长（天数）
E = df['SurvivalStatus']  
groups = df['test']
ix = (groups == 'low')

plt.figure(figsize=(6,4.5))
kmf.fit(T[~ix], E[~ix], label='HOPX-positive')
ax = kmf.plot_survival_function()  #有置信区间
#print("High, Median Survival Time (year):", kmf.median_survival_time_)
kmf.fit(T[ix], E[ix], label='HOPX-negative')
ax = kmf.plot_survival_function(ax=ax)
#print("Low, Median Survival Time (year):", kmf.median_survival_time_)
#plt.title('HOPX',fontweight='bold')
plt.xlabel('Time since diagnosis (year)')
plt.ylabel('Overall survival')
plt.text(0,0.155,"p(log-rank test)=0.0026")
plt.text(-1.2,1.1,"d",fontweight='bold',fontsize=20)
plt.title('Radiogenomic signature (2 image features)',fontweight='bold',fontsize=13)
#plt.text(-1.5,1.04,"b",fontweight='bold',fontsize=18)


T_exp, E_exp = df.loc[ix, 'Duration'], df.loc[ix, 'SurvivalStatus']
T_con, E_con = df.loc[~ix, 'Duration'], df.loc[~ix, 'SurvivalStatus']
results = logrank_test(T_exp, T_con, event_observed_A=E_exp, event_observed_B=E_con)
results.print_summary()
print(results.p_value)        

plt.savefig('/path to save KM curves.png/',dpi=800,bbox_inches='tight')